[View in Colaboratory](https://colab.research.google.com/github/saitota/ProgrammingKnock100/blob/master/Chapter04_Q30_Q39.ipynb)

# :muscle: 前書き
詳細は[第1章](https://qiita.com/saitotak/items/73852ecb7cc4b10b42a5)参照、引き続き言語処理100本ノックの第3章を回答していきます。本章では TBD


# :muscle: 第4章: 形態素解析
> 夏目漱石の小説『吾輩は猫である』の文章（[neko.txt](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/neko.txt)）をMeCabを使って形態素解析し，その結果をneko.txt.mecabというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．
> なお，問題37, 38, 39は[matplotlib](http://matplotlib.org/)もしくは[Gnuplot](http://www.gnuplot.info/)を用いるとよい．

UTF-8,LF の1万行程度の日本語プレーンテキストでした。形態素解析と統計のライブラリを存分に使うことになりそうな感じです。

```text:neko.txt
一

　吾輩は猫である。
名前はまだ無い。

　どこで生れたかとんと見当がつかぬ。
何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。
（以下略）
```

## ⚾ 30. 形態素解析結果の読み込み
> 形態素解析結果（neko.txt.mecab）を読み込むプログラムを実装せよ．ただし，各形態素は表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をキーとするマッピング型に格納し，1文を形態素（マッピング型）のリストとして表現せよ．第4章の残りの問題では，ここで作ったプログラムを活用せよ．


## 下準備として
- ファイルダウンロード
- janomeのインストール

In [1]:
import urllib.request
urllib.request.urlretrieve('http://www.cl.ecei.tohoku.ac.jp/nlp100/data/neko.txt','./neko.txt')


('./neko.txt', <http.client.HTTPMessage at 0x7f0fef752fd0>)

In [2]:
!pip install janome

    100% |████████████████████████████████| 20.0MB 2.6MB/s 
  Running setup.py bdist_wheel for janome ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /content/.cache/pip/wheels/53/60/be/fe884e2d0ebc9fec0988736cf08a2820ab34e3569fc0c5a25a
Successfully built janome


### 解0-1 janomeによる形態素解析と、neko.txt.janome のテキスト作成（おそい）
- MeCab ではなく janome をインストールします、なぜならインストールが簡単だったから。
- `janome.tokenizer` を使って形態素解析します
    - [janome package — janome.tokenizer](http://mocobeta.github.io/janome/api/janome.html#module-janome.tokenizer)
- やたら遅かったので、開始・終了 時間を計測するコードを入れました
- 実測：42sec


In [3]:
from janome.tokenizer import Tokenizer
import janome
import datetime

print('start:' + str(datetime.datetime.now()))

t = Tokenizer()

# WriteのたびにOpen
with open ('./neko.txt','r',encoding='utf-8') as file_temp:
    for line in file_temp:
        # 形態素解析して保存
        for token in t.tokenize(line, stream=True):
            with open ('./neko.txt.janome-01','a',encoding='utf-8') as file_janome:
                    file_janome.write(str(token) + '\n')

print('end:' + str(datetime.datetime.now()))

'''
一	名詞,数,*,*,*,*,一,イチ,イチ
吾輩	名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
猫	名詞,一般,*,*,*,*,猫,ネコ,ネコ
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
ある	助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル
。	記号,句点,*,*,*,*,。,。,。
名前	名詞,一般,*,*,*,*,名前,ナマエ,ナマエ
（以下略）
'''


start:2018-05-25 12:17:50.457529
end:2018-05-25 12:18:28.796007


'\n一\t名詞,数,*,*,*,*,一,イチ,イチ\n吾輩\t名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ\nは\t助詞,係助詞,*,*,*,*,は,ハ,ワ\n猫\t名詞,一般,*,*,*,*,猫,ネコ,ネコ\nで\t助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ\nある\t助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル\n。\t記号,句点,*,*,*,*,。,。,。\n名前\t名詞,一般,*,*,*,*,名前,ナマエ,ナマエ\n（以下略）\n'

### 解0-2 高速化
- メモリ枯渇を鑑み 形態素解析→write を繰り返すようにしてみましたが、やたらと時間がかかったため、一括で書き込むようにして高速化を図ります
- 実測：29sec

In [4]:
print('start:' + str(datetime.datetime.now()))

t = Tokenizer()

# Openしてから
with open ('./neko.txt','r',encoding='utf-8') as file_temp , open ('./neko.txt.janome-02','a',encoding='utf-8') as file_janome:
    for line in file_temp:
        # 形態素解析して保存
        for x in t.tokenize(line, stream=True):
            file_janome.write(str(x) + '\n') 
            

print('end:' + str(datetime.datetime.now()))

start:2018-05-25 12:18:29.516532
end:2018-05-25 12:18:56.441672


### 解0-3 さらに高速化
- それでも遅い気がしたので、書き込み部分をリスト内包表記にしました、多少早くなった
- 実測：27sec


In [5]:
print('start:' + str(datetime.datetime.now()))

t = Tokenizer()

# 一部リスト内包表記
with open ('./neko.txt','r',encoding='utf-8') as file_temp , open ('./neko.txt.janome-03','a',encoding='utf-8') as file_janome:
    for line in file_temp:
        # 形態素解析して保存
        [file_janome.write(str(x) + '\n') for x in t.tokenize(line, stream=True)]
            

print('end:' + str(datetime.datetime.now()))

start:2018-05-25 12:18:57.195272
end:2018-05-25 12:19:25.374264


### 解0-4 さらに高速化・・・失敗
- 読みにくいですがぜんぶリスト内包表記にしました
- 実測：88sec
  - 遅くなった何故。。。

In [6]:
print('start:' + str(datetime.datetime.now()))

t = Tokenizer()

# リスト内包表記
with open ('./neko.txt','r',encoding='utf-8') as file_temp , open ('./neko.txt.janome-04','a',encoding='utf-8') as file_janome:
    [
        [
            file_janome.write(str(x) + '\n')
            for x in t.tokenize(y, stream=True)
        ]
        for y in file_temp
    ]

print('end:' + str(datetime.datetime.now()))

start:2018-05-25 12:19:26.014804
end:2018-05-25 12:19:53.098418


### 解1 neko.txt.janome のテキスト作成
-  形態素はタブ区切り、品詞等の分類はカンマ区切りなので成型します

In [7]:
with open ('./neko.txt.janome-04','r',encoding='utf-8') as file_temp:
    text = file_temp.readlines()

lst_janome=[]
for line in text:
    lst_janome.append(
        { 'surface': line.split('\t')[0], # 表層形
          'pos'    : line.split('\t')[1].split(',')[0], # 品詞
          'pos1'   : line.split('\t')[1].split(',')[1], # 品詞細分類1
          'base'   : line.split('\t')[1].split(',')[6]   # 基本形
        })

for i in range(10):
    print(lst_janome[i])
  

{'surface': '一', 'pos': '名詞', 'pos1': '数', 'base': '一'}
{'surface': '吾輩', 'pos': '名詞', 'pos1': '代名詞', 'base': '吾輩'}
{'surface': 'は', 'pos': '助詞', 'pos1': '係助詞', 'base': 'は'}
{'surface': '猫', 'pos': '名詞', 'pos1': '一般', 'base': '猫'}
{'surface': 'で', 'pos': '助動詞', 'pos1': '*', 'base': 'だ'}
{'surface': 'ある', 'pos': '助動詞', 'pos1': '*', 'base': 'ある'}
{'surface': '。', 'pos': '記号', 'pos1': '句点', 'base': '。'}
{'surface': '名前', 'pos': '名詞', 'pos1': '一般', 'base': '名前'}
{'surface': 'は', 'pos': '助詞', 'pos1': '係助詞', 'base': 'は'}
{'surface': 'まだ', 'pos': '副詞', 'pos1': '助詞類接続', 'base': 'まだ'}



## ⚾ 31. 動詞
> 動詞の表層形をすべて抽出せよ．

## 解1 普通にループ

In [8]:
lst_res = []
for janome in lst_janome:
    if janome.get('pos') == '動詞':
        lst_res.append(janome.get('surface'))

print(lst_res[::10])    

['生れ', '煮', '見', 'し', '坐っ', '助から', '付い', 'られ', '見', '渡っ', '廻り', 'もぐり込ん', '忍び込ん', '考える', 'ねぶっ', 'れ', '云う', '来', '眺め', 'し', 'いう', '帯び', '垂らす', '出来', 'られ', '得る', 'やむを得ん', '割り込む', 'くる', '至っ', '怒っ', '来たそ', '隣り', 'いる', 'すまし', '待つ', 'し', 'おら', '吹き出す', 'やめ', 'やっ', '感ずる', '飛ぶ', 'ある', '来', '見', '執っ', 'する', 'れ', '交ぜ', '思っ', 'なっ', '打ち', '知ら', 'くれ', 'する', '養う', '心付か', '抛', 'ある', 'いる', 'し', 'せる', 'する', 'いる', '生じ', '見える', '見違える', 'なる', '吐く', 'とっ', 'とろ', 'し', 'し', '引き受ける', '這い', 'え', 'がっ', 'する', 'し', 'とっ', '儲け', 'なっ', 'い', '見え', '云う', 'する', 'せ', '得る', 'いる', 'なっ', 'ある', '力め', '出さ', '信じよ', 'いる', 'なる', 'おっ', '知ら', 'い', '笑っ', 'いる', 'いる', 'ある', 'なっ', 'こぼし', '立て', '帰る', '取ら', 'いる', '見', '持っ', '思っ', '思い', '見', '云う', '得る', '出来', '這入っ', 'かかわら', 'いう', 'し', 'わかる', 'し', 'ある', 'なっ', 'あれ', 'れ', '極め', 'なれ', 'いる', '求め', 'いる', '引張る', '食い', 'なら', 'やり', '思い', '見る', '分ら', 'せ', 'もし', '出る', '透かし', '換え', '思う', 'し', 'いる', 'あっ', 'とっ', 'いる', 'しゃくい', 'いる', 'い', '食い', '濁っ', 'ある', '利く', 'し', '上っ', '突き付け', '行っ', 'い', 'いる', '剃っ', '

## 解2. 内容表記と重複排除
- このくらいのループであればリスト内包表記のほうがいいかも
- `set()` による重複排除

In [9]:
lst_res = list(set([x.get('surface') for x in lst_janome if x.get('pos') == '動詞']))
print(lst_res[::10])

['越す', '略する', '焚く', 'なれる', '増し', '這い', '鳴き', '詰め込む', '出掛け', '止す', 'おひゃらかす', 'しから', '持て余す', '儲ける', '書き付け', '聞き糺し', '構え', '憤っ', '称え', '直し', '破る', '傾け', 'あやまる', '見付け', '焦げ', '揚げる', '擦', '試みよ', '舞わ', '思い出せ', '整っ', '分りゃ', '争う', '睡ら', '流行っ', '下げ', '装っ', '魂消る', '雇っ', '喜ぶ', '真似れ', '煮え', '寝込ん', '笑う', '使い', '隠れる', '草臥れ', '用い', 'ねる', 'のせ', '帰る', '売り飛ばし', '避け', '間に合わ', '及び', '来よ', '切る', '割り切れ', '畏まっ', '制さ', 'かため', 'すっ', 'ここ', '吹き出す', '捕れる', '向っ', '思い立ち', '致し', '卸せ', '欠く', '重ねる', '暮らさ', '疑っ', '考え直し', '参ろ', '変ずる', '飛び込も', '引張る', '問う', '罵っ', '覗く', '驚かさ', '消え去っ', 'すかし', '較べ', 'うたう', '顧みる', '問い', '突き立て', '振い落し', '積ま', '有し', '戻る', 'たて', '降りよ', '騰', '下がる', '寝れ', '結え', 'あまる', '歩く', '尽くさ', '言い付け', '濁す', '冷かし', '放さ', 'しょ', 'すまい', 'なさい', '洩らす', '隠し', '乗りゃ', '敲い', '途切れ', '目立つ', '通ずる', '掘り出す', '捧ぐ', '動ずる', '聞け', '蒙ろ', '過ごす', 'ありゃ', '吹きかける', '始める', '上がろ', '直さ', 'かかっ', '落ちつか', '伺お', '減ら', 'われ', 'ねむり', '砕い', '並べ', '逸する', '附け', '動じ', '仰ぎ', '取り寄せる', '貰っ', '尖', '属し', '踏ま', 'からかお', '表する', 'ちょろまかし', '逝い', '突い', '願い', 


## ⚾ 32. 動詞の原形
> 動詞の原形をすべて抽出せよ．


## 解1. 内包表記
- 原型=基本形（base）のようです

In [10]:
st_res = list(set([x.get('base') for x in lst_janome if x.get('pos') == '動詞']))
print(lst_res[::10])

['越す', '略する', '焚く', 'なれる', '増し', '這い', '鳴き', '詰め込む', '出掛け', '止す', 'おひゃらかす', 'しから', '持て余す', '儲ける', '書き付け', '聞き糺し', '構え', '憤っ', '称え', '直し', '破る', '傾け', 'あやまる', '見付け', '焦げ', '揚げる', '擦', '試みよ', '舞わ', '思い出せ', '整っ', '分りゃ', '争う', '睡ら', '流行っ', '下げ', '装っ', '魂消る', '雇っ', '喜ぶ', '真似れ', '煮え', '寝込ん', '笑う', '使い', '隠れる', '草臥れ', '用い', 'ねる', 'のせ', '帰る', '売り飛ばし', '避け', '間に合わ', '及び', '来よ', '切る', '割り切れ', '畏まっ', '制さ', 'かため', 'すっ', 'ここ', '吹き出す', '捕れる', '向っ', '思い立ち', '致し', '卸せ', '欠く', '重ねる', '暮らさ', '疑っ', '考え直し', '参ろ', '変ずる', '飛び込も', '引張る', '問う', '罵っ', '覗く', '驚かさ', '消え去っ', 'すかし', '較べ', 'うたう', '顧みる', '問い', '突き立て', '振い落し', '積ま', '有し', '戻る', 'たて', '降りよ', '騰', '下がる', '寝れ', '結え', 'あまる', '歩く', '尽くさ', '言い付け', '濁す', '冷かし', '放さ', 'しょ', 'すまい', 'なさい', '洩らす', '隠し', '乗りゃ', '敲い', '途切れ', '目立つ', '通ずる', '掘り出す', '捧ぐ', '動ずる', '聞け', '蒙ろ', '過ごす', 'ありゃ', '吹きかける', '始める', '上がろ', '直さ', 'かかっ', '落ちつか', '伺お', '減ら', 'われ', 'ねむり', '砕い', '並べ', '逸する', '附け', '動じ', '仰ぎ', '取り寄せる', '貰っ', '尖', '属し', '踏ま', 'からかお', '表する', 'ちょろまかし', '逝い', '突い', '願い', 

## ⚾ 33. サ変名詞
> サ変接続の名詞をすべて抽出せよ．

## 解1. 内包表記
- 品詞細分類1がサ変接続のものを探します

In [11]:
lst_res = list(set([x.get('base') for x in lst_janome if x.get('pos1') == 'サ変接続']))
print(lst_res[::10])

['相伝', '居住', '想見', 'ごろごろ', '逗留', '大悟', '比例', '遊戯', '周旋', '襲撃', '発覚', '満足', '鼓舞', '代理', '演奏', '御馳走', '失敬', '接待', '遂行', '承認', '身動き', '拝顔', '遠慮', '寄附', '欠伸', '保証', '起因', '解脱', '苦悶', '出ろ', '批評', '制限', '交際', '賛同', '伴奏', '自白', '下落', '快走', '作用', '象徴', '関', '記録', '傑出', '監督', '苦難', '懺悔', '損害', '所有', '仮定', '焦慮', '散歩', '養成', '退校', '連署', '支配', '話', '挑撥', '一周', '乱入', '服薬', '清聴', '悪戯', '弁護', '闖入', '屈従', '往来', '対面', '贋造', '騒動', '全快', '位置', '巡回', '逃亡', '前進', '興奮', '出現', '網羅', '豹変', '伝来', '著述', '明記', '相違', '激', '尋問', '際会', '予言', '翻訳', '謝罪', '成道', '沈思', '演習', '保護', '合図', '早合点', '厳命', '在宿', '告訴', 'ていかん', '通信', '陳列', '綜合', '微動', '散会', '揶揄', '寂滅', '披瀝', '苦心', '架', '諒察', '宙返り', '本復', '拝借', '放擲', '安心', '——」', '晴れ晴れ', '精読', '伝染', '畏怖', '再興', '是非', '使役', '露見', '容喙', '免職', '奉行', '蓄積', '合奏']


## ⚾ 34. 「AのB」
> 2つの名詞が「の」で連結されている名詞句を抽出せよ．

## 解1. 普通にループ
- 名詞 の 名詞 を探します

In [12]:
lst_res = []
for i,janome in enumerate(lst_janome):
    if janome.get('base') == 'の':
        if (lst_janome[i-1].get('pos') == '名詞') and (lst_janome[i+1].get('pos') == '名詞'):
          lst_res.append(lst_janome[i-1].get('base') + janome.get('base') + lst_janome[i+1].get('base'))

print(lst_res[::20])

['彼の掌', '彼の書生', '主人の傍', '家の書生', '今更のよう', '主人の彩色', '純粋の黒', '吾輩の心臓', '彼の鼻', '黒の子分', '元の通り', 'ハリソンの歴史', '赤松の間', '主人のよう', '吾輩の主人', '門の格子', '実のところ', '後の猫', '壺の中', '障子の隙', '主人のよう', '主人の心', '二絃琴の師匠', '贅沢の結果', '二の真理', '在来の通り', '欣羨の意', '理詰の虚言', '初春の長閑', '主人の笑い声', 'つもりのところ', '方面の研究', '娼家の下婢', '念の体', '男の事', '感謝の意', '世紀の今日', '自分の境遇', '毛の病気', '平等の水彩', 'トチメンボーの復讐', '明治の文壇', '他の松', '博士の夫人', '遠くの方', '門の内', '水の労', '縮緬の羽織', '主人の我儘', '障子のうち', '教師の所', '吾輩の写真', '居士の事', '自分の家', '先刻の不平', '冊の内', '一流の論理', '迷亭の専断', '縄の一端', '御存じの平均', '埒の方', '女の声', '金田の倉', '私の家', '寒月の方', '車屋の神', '地球の磁気', '枚の絵葉書', '月並の標本', 'ステッキの代り', '活眼の士', '尻尾の先', '無用の長物', '車夫の声', '夜光の明', 'さんの話', '長吉の方', '尻尾の先', '忠告の通り', '垣の隙', '護の恐れ', '眉の根', '鼻の持主', '何のため', '空の縄張', '雪隠の横', '色の口', '僧の癖', '奴の娘', '天気の日', '右手の指', '供の袖', '時の印象', '鼻の中', '自分のため', '心の故', '下の方', '腹の内', '人の悪口', '本人の随意', '迷亭の食い気', '元のよう', '迷亭の答え', '近来の珍', 'ものの息女', '派の元祖', '智識の問屋', '贔負の尊公', '雪の夜', '目下の状態', '鼠の糞', '狼藉の練', '柳行李の間', '俗人の考', '失敗の痕', '神の製作', '未来の事件', '柳行李の辺', '山の芋の箱', '寝室の方', '

## ⚾ 35. 名詞の連接
> 名詞の連接（連続して出現する名詞）を最長一致で抽出せよ．


## 解1. 普通にループ
- 名詞 の繰り返しを探します
- 名詞×5以上 を求めてみました

In [13]:
lst_res = []
lst_tmp = []
for janome in lst_janome:
    if janome.get('pos') == '名詞':
         lst_tmp.append(janome.get('surface'))
    else:
        lst_tmp.clear()
    if(len(lst_tmp) > 4): #ほんとは1が条件
        lst_res.append(''.join(lst_tmp))

print(lst_res[::10])

['壱円五十銭', '是非共二三十', '四時十五分前', '八万八千八百八十本', '千七百八十', '帰泉院殿黄鶴', '明治三十八年', '十二円五十', '十年一狐裘', '三日三晩手', '弥次馬兼援兵', '珍野苦沙弥先生虎', '明治十一年九月廿八', '際武右衛門君', '五六十本半紙', '六十二回六十']


## ⚾ 36. 単語の出現頻度
> 文章中に出現する単語とその出現頻度を求め，出現頻度の高い順に並べよ．


## 解1. collections型
- `collections` の `most_common([n])`ｄ集計
    - [8.3. collections](https://docs.python.jp/3/library/collections.html#collections.Counter.most_common)
- Pandasとか使うのかと思った

In [14]:
import collections

lst_tmp = []
for janome in lst_janome:
    lst_tmp.append(janome.get('surface'))

cnt = collections.Counter(lst_tmp)

print(cnt.most_common()[::10])

[('の', 9194), ('で', 3805), ('ん', 1569), ('主人', 932), ('そう', 546), ('…', 433), ('三', 322), ('寒月', 286), ('なく', 258), ('先生', 233), ('など', 205), ('より', 181), ('日', 154), ('だって', 148), ('ため', 140), ('女', 126), ('という', 116), ('さえ', 109), ('なり', 104), ('沙弥', 99), ('下', 94), ('云わ', 89), ('奴', 86), ('独', 83), ('話', 78), ('運動', 74), ('つい', 72), ('見える', 70), ('様', 66), ('八', 65), ('無', 62), ('なさい', 61), ('り', 58), ('鼠', 56), ('るる', 54), ('そこ', 51), ('裏', 49), ('しきりに', 48), ('向う', 46), ('不思議', 45), ('上っ', 44), ('近頃', 44), ('答え', 42), ('結果', 41), ('ことに', 40), ('当人', 39), ('わから', 39), ('いけ', 38), ('事実', 37), ('よかろ', 36), ('面', 36), ('さすが', 35), ('探偵', 35), ('けれども', 34), ('向っ', 33), ('衛門', 33), ('切っ', 32), ('過ぎる', 32), ('たろ', 31), ('かかる', 30), ('似', 30), ('返', 30), ('質問', 29), ('舌', 29), ('いっしょ', 28), ('起し', 28), ('爪', 27), ('損', 27), ('付い', 26), ('注意', 26), ('先方', 26), ('なん', 25), ('二つ', 25), ('この間', 24), ('ありがたい', 24), ('よっぽど', 24), ('寸', 24), ('やろ', 23), ('説', 23), ('解釈', 23), ('尊敬', 22), ('大抵', 


## ⚾ 37. 頻度上位10語
> 出現頻度が高い10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ．




## ⚾ 38. ヒストグラム
> 単語の出現頻度のヒストグラム（横軸に出現頻度，縦軸に出現頻度をとる単語の種類数を棒グラフで表したもの）を描け．



## ⚾ 39. Zipfの法則
> 単語の出現頻度順位を横軸，その出現頻度を縦軸として，両対数グラフをプロットせよ．
